In [3]:
import pandas as pd
import numpy as np
from math import pi
from scipy.integrate import odeint
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# Computing dynamical ellipticity

In [4]:
def rhs_clairaut(y,x,xcmb,rhof,rhom):
    if x < xcmb:
        dydx = np.array([ y[1] , -(6/x)*y[1]])
    else:
        zeta = rhom/(rhom + (rhof-rhom)*(xcmb/x)**3)
        dydx = np.array([ y[1], (-6/x)*zeta*y[1] - 
                         6/(x**2) * (zeta - 1) * y[0]])
    return dydx

In [5]:
def shoot_clairaut_homog(v,mm,rhom,rhof,xcmb):
    y0 = [float(v),0.0]
    xx = np.linspace(xcmb, 1, 201)
    yy = odeint(rhs_clairaut,y0,xx,args=(xcmb,rhof,rhom))
    
    nx = len(xx) - 1
    
    f = yy[nx,1] + 2*yy[nx,0] - 2.5*mm
    return f,yy

In [6]:
def clairaut_homog(mm,rhom,rhof,xcmb):
    v0 = 0.001
    
    def func(x):
        f,yy = shoot_clairaut_homog(x,mm,rhom,rhof,xcmb)
        return f
    
    v = fsolve(func,v0)
    
    f,yy = shoot_clairaut_homog(v,mm,rhom,rhof,xcmb)
    
    ellf = v
    
    flat = yy[-1,0]
    ell = (rhof*xcmb**5*ellf + rhom*(flat-ellf*xcmb**5))/(rhof*xcmb**5 + 
                                                         rhom*(1-xcmb**5))
    return ellf,ell

# Fluid core & whole planet ellipticities 

In [15]:
### Assuming rotation frequency = orbital frequency
day_sec = 86400 #seconds
G = 6.67430 * 10**-11 # m^3 kg^-1 s^-2

R = 6371 * 10**3 #m
Rf = 3485 * 10**3 #m
M = 5.97219 * 10**24 #kg
rhom = 4.5 * 10**3 #kg/m^3
rhof = 13.0 * 10**3 #kg/m^3

rot = 2*pi/(day_sec) #rad/sec
print(rot)

xcmb = Rf/R
mm = (rot**2 * R**3)/(G*M)
ellf,ell = clairaut_homog(mm,rhom,rhof,xcmb)

7.27220521664304e-05


In [14]:
f'{ell[0]:.{4}f}'

'0.0033'